In [55]:
import glob
img = glob.glob('Totalsegmentator_dataset_v201/*/segmentations/')
len(img)
for i in range(len(img)):
    scan = glob.glob(img[i]+'*')
    #print(len(scan))


In [56]:
pwd

'/Users/trl102/Downloads/scan'

In [57]:
cd Totalsegmentator_dataset_v201

/Users/trl102/Downloads/scan/Totalsegmentator_dataset_v201


In [59]:
cd s0001//

/Users/trl102/Downloads/scan/Totalsegmentator_dataset_v201/s0001


In [64]:
cd segmentations/

[Errno 2] No such file or directory: 'segmentations/'
/Users/trl102/Downloads/scan/Totalsegmentator_dataset_v201/s0001/segmentations


In [65]:
ls

adrenal_gland_left.nii.gz            rib_left_4.nii.gz
adrenal_gland_right.nii.gz           rib_left_5.nii.gz
aorta.nii.gz                         rib_left_6.nii.gz
atrial_appendage_left.nii.gz         rib_left_7.nii.gz
autochthon_left.nii.gz               rib_left_8.nii.gz
autochthon_right.nii.gz              rib_left_9.nii.gz
brachiocephalic_trunk.nii.gz         rib_right_1.nii.gz
brachiocephalic_vein_left.nii.gz     rib_right_10.nii.gz
brachiocephalic_vein_right.nii.gz    rib_right_11.nii.gz
brain.nii.gz                         rib_right_12.nii.gz
clavicula_left.nii.gz                rib_right_2.nii.gz
clavicula_right.nii.gz               rib_right_3.nii.gz
colon.nii.gz                         rib_right_4.nii.gz
common_carotid_artery_left.nii.gz    rib_right_5.nii.gz
common_carotid_artery_right.nii.gz   rib_right_6.nii.gz
costal_cartilages.nii.gz             rib_right_7.nii.gz
duodenum.nii.gz                      rib_right_8.nii.gz
esophagus.nii.gz                     rib_right_9.ni

# mask labels

In [35]:
import os 
import pandas as pd 

all_scans =os.listdir('Totalsegmentator_dataset_v201/s1391/segmentations/')
all_scans.sort()
all_scans
df = pd.DataFrame(all_scans)
df.to_excel('mask_label.xlsx')

In [ ]:
Totalsegmentator_dataset_v201/s0003/ct.nii.gz

# Devide the datset based on split column value 'train' or 'test'



In [10]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_excel('meta.xlsx')
# select only train values in split column
df = df[df['split']=='train']
print(df.columns)
# make directories for train and test
import os
os.makedirs('dataset_nii/train/images', exist_ok=True)
os.makedirs('dataset_nii/train/labels', exist_ok=True)
os.makedirs('dataset_nii/test/images', exist_ok=True)
os.makedirs('dataset_nii/test/labels', exist_ok=True)

# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
image_folder = 'New'
label_folder = 'segmentations' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = 'New/'+files[i]+'/ct.nii.gz'
    labels = 'New/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/train/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/train/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'split',
       'manufacturer', 'scanner_model', 'kvp', 'pathology',
       'pathology_location'],
      dtype='object')


100%|██████████| 1082/1082 [00:20<00:00, 52.71it/s]


In [11]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_excel('meta.xlsx')
# select only train values in split column
df = df[df['split']=='test']
print(df.columns)
# make directories for train and test
import os
os.makedirs('dataset_nii/train/images', exist_ok=True)
os.makedirs('dataset_nii/train/labels', exist_ok=True)
os.makedirs('dataset_nii/test/images', exist_ok=True)
os.makedirs('dataset_nii/test/labels', exist_ok=True)

# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
image_folder = 'New'
label_folder = 'segmentations' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = 'New/'+files[i]+'/ct.nii.gz'
    labels = 'New/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/test/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/test/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'split',
       'manufacturer', 'scanner_model', 'kvp', 'pathology',
       'pathology_location'],
      dtype='object')


100%|██████████| 89/89 [00:02<00:00, 41.85it/s]


In [13]:
print(len('datase_nii/train/images/') == len('datase_nii/train/labels/'), len('datase_nii/test/images/') == len('datase_nii/test/labels/'))

True True
